# Scrape Report URLs

#### Imports

In [73]:
import requests
import pyodbc
from bs4 import BeautifulSoup
from sqlalchemy import create_engine,text,String
from io import BytesIO
import pandas as pd
import csv
import zipfile
import io
import os

In [74]:
url = 'https://insights.stackoverflow.com/survey'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
current_tags = soup.find_all('div', class_= 'copy-container')
target_divs = soup.find_all('div' , class_= 'past-years' )

# Grab latest Report Url

In [75]:
# For current year
current_url = []
for current in current_tags:
    anchor_element = current.find_all('a')
    
    for anchor in anchor_element:
        href_value = anchor.get("href")
        if href_value.endswith(".zip"):
            current_url.append(href_value)

In [76]:
current_url

['https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip']

# Getting list of all URLs

In [77]:
urls = []

# for previous years

for current in current_tags:
    anchor_element = current.find_all('a')
    
    for anchor in anchor_element:
        href_value = anchor.get("href")
        if href_value.endswith(".zip"):
            current_url.append(href_value)
        else:
            "Check data structure"

# For Previous years
# Loop through each target div
for target_div in target_divs:
    # Find all anchor elements under the target div
    anchor_elements = target_div.find_all("a")

    # Extract and print href values
    for anchor in anchor_elements:
        href_value = anchor.get("href")
        if href_value.endswith(".zip"):
            urls.append(href_value)

all_urls = current_url + urls

# all_urls.pop(0)

In [79]:
all_urls.pop(0)


'https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip'

In [91]:
all_urls=all_urls[0:5]
all_urls

['https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip',
 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip',
 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2021.zip',
 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2020.zip',
 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2019.zip']

### Using list iteration of urls to download zip file, unzip and select the required csv file.

In [92]:
# downloads zip file
zip_downloads = []
def download_zip_file(zip_url):
    response = requests.get(zip_url)
    if response.status_code == 200:
        zip_file = response.content
        zip_downloads.append(zip_file)
        return response.content
    else:
        return None
    


In [93]:
# Unpacks content of zip file
file_names = []
def view_zip_contents(zip_data):
    with zipfile.ZipFile(BytesIO(zip_data), 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            file_names.append(file_name)

In [94]:
def extract_characters_before_zip(url):
    # Find the position of '.zip' in the URL
    zip_index = url.find('.zip')
    # Check if '.zip' was found in the URL
    if zip_index != -1:
        # Extract the 4 characters before '.zip'
        four_chars_before_zip = url[zip_index - 4:zip_index]
        return four_chars_before_zip
    else:
        return None
    
extract_characters_before_zip('https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip')

'2022'

In [95]:
# Extract all csv files from unpacked zip folder
csv_files = []

def extract_csv_from_zip(zip_data, target_csv_name):
    csv_data = None
    with zipfile.ZipFile(io.BytesIO(zip_data), 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.lower().endswith('.csv') and target_csv_name in file_name:
                with zip_ref.open(file_name) as csv_file:
                    csv_data = pd.read_csv(csv_file, dtype=str, low_memory=False,encoding='utf-8')
                    csv_files.append(csv_data)
                break
    return csv_data 


In [96]:
# Passing list urls and downloaded zip files which were placed in a list
for i in range(len(all_urls)):
    download_zip_file(all_urls[i])
    view_zip_contents(zip_downloads[i])


In [97]:
# Remove all unwanted files
file_names = [ex for ex in file_names if ex.endswith('.csv') and 'MAC' not in ex and 'survey_results_schema'  not in ex and '._' not in ex]
file_names = [ f.split('/')[-1] for f in file_names]
file_names

['survey_results_public.csv',
 'survey_results_public.csv',
 'survey_results_public.csv',
 'survey_results_public.csv',
 'survey_results_public.csv']

## Extracted csv files from all years in a list

In [98]:
for i in range(len(file_names)):
    extract_csv_from_zip(zip_downloads[i],file_names[i])

## Key value pair to identify year of dataset {'year':'dataset'}

In [107]:
dic = {}
year = int(extract_characters_before_zip(all_urls[-1]))
print(year)
temp = len(csv_files)
# print(temp)


while temp > 0:
    dic[int(year)] = csv_files[temp-1]
    temp -= 1
    year += 1

2019


In [108]:
# print(dic.items())

## Write datasets to control DB as tables for each year

In [109]:
# # Hosted SQLServer DB instance
# server = 'data-science.ck0dwzfc16xf.eu-west-1.rds.amazonaws.com'
# database = ''
# username = ''
# password = ''

# Connection parameters
server_name = "localhost\MYSQL2019" 
database_name = ""  
username = ""  
password = ""  

In [110]:
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"

# Create an SQL Server engine
engine = create_engine(connection_string)

In [111]:
# from sqlalchemy.exc import SQLAlchemyError
# try:
#     # Try executing a simple query to test the connection
#     result = engine.execute('SELECT 1')
#     print("Connection successful!")
#     result.close()  # Close the result set
# except SQLAlchemyError as e:
#     print("Connection failed:", e)

In [112]:
# Write all year datasets to control DB while truncating column names that exceed the limit of 128 characters
for k,v in dic.items():
    new_columns = [col[:128] if len(col) > 128 else col for col in v.columns]
    v.columns = new_columns
    dtype_mapping = {col: String for col in v.select_dtypes(include=['object']).columns}
    v.to_sql("dev_survey_"+str(k), engine, if_exists='replace', index=False,dtype=dtype_mapping)
    print(str(k)+" survey dataset")

2019 survey dataset
2020 survey dataset
2021 survey dataset
2022 survey dataset
2023 survey dataset


In [ ]:
from datetime import datetime, timedelta
current_year = datetime.now().year
print(type(str(current_year)))